In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
 
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from keras.layers import Conv2D, UpSampling2D
from skimage.color import rgb2lab, lab2rgb
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import Input, Reshape, concatenate
from keras.models import Model
from keras.layers.core import RepeatVector
from skimage.transform import resize
 
import numpy as np
import os
import cv2



In [ ]:
# Get images
X = []
#path ="/content/drive/MyDrive/colorizing/Dataset/"
path ="val2017/"
i = 0
for filename in os.listdir(path):
    i+=1
    img = cv2.imread(path+filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(256,256))
    X.append(img)
    if i == 500 :
      break


X = np.array(X, dtype=float)


In [ ]:
print(X.shape)

(500, 256, 256, 3)


In [ ]:
Xtrain = rgb2lab(1.0/255*X)[:,:,:,0]
Ytrain = rgb2lab(1.0/255*X)[:,:,:,1:]
Xtrain = Xtrain = Xtrain.reshape(Xtrain.shape+(1,))
Ytrain /= 128

In [ ]:
print(Xtrain.shape)

(500, 256, 256, 1)


loss curve

In [ ]:
class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))

**Train model **

In [ ]:
#Load weights
inception = tf.keras.applications.InceptionResNetV2(include_top=True, weights='imagenet')


#Encoder
encoder_input = Input(shape=(256, 256, 1,))
encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(encoder_input)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(encoder_output)
encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(encoder_output)

#Fusion
embed_input = Input(shape=(1000,))
fusion_output = RepeatVector(32 * 32)(embed_input) 
fusion_output = Reshape(([32, 32, 1000]))(fusion_output)
fusion_output = concatenate([encoder_output, fusion_output], axis=3) 
fusion_output = Conv2D(256, (1, 1), activation='relu', padding='same')(fusion_output) 

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)

decoder_output = UpSampling2D((2, 2))(decoder_output)

decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)

decoder_output = UpSampling2D((2, 2))(decoder_output)

decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)

decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[encoder_input, embed_input], outputs=decoder_output)
        
def create_inception_embedding(grayscaled_rgb): # 265 *265 * 1 
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        i = lab2rgb(i)
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)

    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    #with inception.graph.as_default():

    embed = inception.predict(grayscaled_rgb_resized)
    
    return embed

x=[Xtrain, create_inception_embedding(Xtrain)]
print(model.summary())


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d_14 (Conv2D)              (None, 128, 128, 64) 640         input_3[0][0]                    
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 128, 128, 128 73856       conv2d_14[0][0]                  
__________________________________________________________________________________________________
conv2d_16 (Conv2D)              (None, 64, 64, 128)  147584      conv2d_15[0][0]                  
____________________________________________________________________________________________

In [ ]:
#Train model
model.compile(optimizer='adam',loss='mse')
#history = LossHistory()

model.fit(x=x, y=Ytrain, epochs=10  )

#, batch_size=10 ,callbacks=[history]
#Save model
model.save('/content/drive/MyDrive/colorizing/model.h5')



Epoch 1/10
16/16 [==============================] - 49s 1s/step - loss: 0.1559
Epoch 2/10
16/16 [==============================] - 10s 651ms/step - loss: 0.0123
Epoch 3/10
16/16 [==============================] - 10s 652ms/step - loss: 0.0129
Epoch 4/10
16/16 [==============================] - 10s 652ms/step - loss: 0.0123
Epoch 5/10
16/16 [==============================] - 10s 650ms/step - loss: 0.0119
Epoch 6/10
16/16 [==============================] - 10s 650ms/step - loss: 0.0129
Epoch 7/10
16/16 [==============================] - 10s 651ms/step - loss: 0.0124
Epoch 8/10
16/16 [==============================] - 10s 653ms/step - loss: 0.0121
Epoch 9/10
16/16 [==============================] - 10s 652ms/step - loss: 0.0122
Epoch 10/10
16/16 [==============================] - 10s 650ms/step - loss: 0.0131


In [ ]:
results = model.evaluate(x=x, Ytrain, batch_size=128)


SyntaxError: ignored

In [ ]:
print(len(x[1]))

500




test accurcy

***Pridict***

In [ ]:
from keras.models import load_model
from skimage.color import rgb2lab, lab2rgb
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from skimage.transform import resize
from skimage.io import imsave
import tensorflow as tf
import numpy as np
import cv2
import os

#Load model
model = load_model('/content/drive/MyDrive/colorizing/modelFamily.h5')
#Load weights
inception = tf.keras.applications.InceptionResNetV2(include_top=True, weights='imagenet')

def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        i = lab2rgb(i)
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    embed = inception.predict(grayscaled_rgb_resized)
    return embed

color_me = []
path ="/content/drive/MyDrive/colorizing/Test/People/"
for filename in os.listdir(path):
    img = cv2.imread(path+filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(256,256))
    color_me.append(img)
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict([color_me, create_inception_embedding(color_me)])
output *= 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("img"+str(i)+".png", lab2rgb(cur))

In [ ]:
import numpy as np
import cv2
from skimage.color import rgb2lab, lab2rgb, rgb2gray, xyz2lab
from skimage.io import imsave
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

def PSNR(original , colored ):
	mse = np.mean((original - colored)**2)
	print(mse.shape)
	if(mse == 0 ): 
		return 100 
	max_pixel=255.0
	psnr = 20 *np.log10(max_pixel/ np.sqrt(mse)) 
	return psnr

ORIGINAL = img_to_array(load_img('/content/drive/MyDrive/colorizing/Dataset/colour.jpg'))
ORIGINAL = np.array(ORIGINAL, dtype=float)
ORIGINAL = cv2.resize(ORIGINAL,(256,256))

COLOR = img_to_array(load_img('img3.png'))
COLOR = np.array(COLOR, dtype=float)

psnr =PSNR(ORIGINAL,COLOR)

print(psnr) 

()
9.256170863109215


In [ ]:

print(history.losses)

NameError: ignored

loss curve
